In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
from math import atan2, pi, sqrt, cos, sin, floor
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)  
from tensorflow.keras.layers import Dense, MaxPooling1D, Flatten
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.compat.v1.keras.backend as K
from tensorflow.keras.models import load_model
from tcn import TCN, tcn_full_summary
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform
from keras_flops import get_flops
import pickle
import csv
import random
import itertools
import math
import time
from dataset2.data_utils_2 import *
from error_metrics import *

In [ ]:
window_size = 100
stride = 20

f = 'dataset2/'
X_train,Y_Pos_train, Physics_Vec_train, x_vel_train, y_vel_train, x0_list_train, y0_list_train, size_of_each_train = import_agrobot_dataset_p3(dataset_folder=f, type_flag=1, window_size=window_size, stride=stride)
P = np.repeat(Physics_Vec_train,window_size).reshape((Physics_Vec_train.shape[0],window_size,1))
X_train = np.concatenate((X_train,P),axis=2)

X_test,Y_Pos_test, Physics_Vec_test, x_vel_test, y_vel_test, x0_list_test, y0_list_test, size_of_each_test= import_agrobot_dataset_p3(type_flag = 2, dataset_folder=f,window_size=window_size, stride=stride)
P_test = np.repeat(Physics_Vec_test,window_size).reshape((Physics_Vec_test.shape[0],window_size,1))
X_test = np.concatenate((X_test,P_test),axis=2)

In [ ]:
nb_filters = 32
kernel_size = 5
dilations = [1,2,4,8,16,32,64,128]
dropout_rate = 0.0
use_skip_connections = True

batch_size, timesteps, input_dim = 256, window_size, X_train.shape[2]
i = Input(shape=(timesteps, input_dim))

m = TCN(nb_filters=nb_filters,kernel_size=kernel_size,dilations=dilations,dropout_rate=dropout_rate,
            use_skip_connections=use_skip_connections)(i)  

m = tf.reshape(m, [-1, nb_filters, 1])
m = MaxPooling1D(pool_size=(2))(m)
m = Flatten()(m)
m = Dense(32, activation='linear', name='pre')(m)
output1 = Dense(1, activation='linear', name='velx')(m)
output2 = Dense(1, activation='linear', name='vely')(m)
model = Model(inputs=[i], outputs=[output1, output2])
opt = tf.keras.optimizers.Adam()
model.compile(loss={'velx': 'mse','vely':'mse'},optimizer=opt)  
model.summary()

In [ ]:
model_name = 'Agrobot_Second_TCN.hdf5'
checkpoint = ModelCheckpoint(model_name, monitor='loss', verbose=1, save_best_only=True)
model.fit(x=X_train, y=[x_vel_train, y_vel_train],epochs=900, shuffle=True,callbacks=[checkpoint],batch_size=batch_size)     

In [ ]:
model = load_model('Agrobot_Second_TCN.hdf5',custom_objects={'TCN':TCN})

In [ ]:
a = 0
b = size_of_each_test[0]
ATE = []
RTE = []
ATE_dist = []
RTE_dist = []
for i in range(len(size_of_each_test)):
    X_test_sel = X_test[a:b,:,:]
    x_vel_test_sel = x_vel_test[a:b]
    y_vel_test_sel = y_vel_test[a:b]
    if(i!=len(size_of_each_test)-1):
        a += size_of_each_test[i]
        b += size_of_each_test[i]

    y_pred = model.predict([X_test_sel])

    pointx = []
    pointy = []
    Lx =  x0_list_test[i]
    Ly = y0_list_test[i]
    for j in range(len(x_vel_test_sel)):
        Lx = Lx + (x_vel_test_sel[j]/(((window_size-stride)/stride)))
        Ly = Ly + (y_vel_test_sel[j]/(((window_size-stride)/stride)))    
        pointx.append(Lx)
        pointy.append(Ly)   
    Gvx = pointx
    Gvy = pointy
    
    pointx = []
    pointy = []
    Lx =  x0_list_test[i]
    Ly = y0_list_test[i]
    for j in range(len(x_vel_test_sel)):
        Lx = Lx + (y_pred[0][j]/(((window_size-stride)/stride)))
        Ly = Ly + (y_pred[1][j]/(((window_size-stride)/stride)))
        pointx.append(Lx)
        pointy.append(Ly)
    Pvx = pointx
    Pvy = pointy    
    
    at, rt, at_all, rt_all = Cal_TE(Gvx, Gvy, Pvx, Pvy,
                                    sampling_rate=100,window_size=window_size,stride=stride)
    ATE.append(at)
    RTE.append(rt)
    ATE_dist.append(Cal_len_meters(Gvx, Gvy))
    RTE_dist.append(Cal_len_meters(Gvx, Gvy, 600))
    print('ATE, RTE, Trajectory Length, Trajectory Length (60 seconds)',ATE[i],RTE[i],ATE_dist[i],RTE_dist[i])
    
print('Median ATE and RTE', np.median(ATE),np.median(RTE))

In [ ]:
a = 0
b = 3000

X_test_sel = X_test[a:b,:,:]
x_vel_test_sel = x_vel_test[a:b]
y_vel_test_sel = y_vel_test[a:b]

y_pred = model.predict(X_test_sel)

pointx = []
pointy = []
Lx =  x0_list_test[i]
Ly = y0_list_test[i]
for j in range(len(x_vel_test_sel)):
    Lx = Lx + (x_vel_test_sel[j]/(((window_size-stride)/stride)))
    Ly = Ly + (y_vel_test_sel[j]/(((window_size-stride)/stride)))    
    pointx.append(Lx)
    pointy.append(Ly)   
Gvx = pointx
Gvy = pointy

pointx = []
pointy = []
Lx =  x0_list_test[i]
Ly = y0_list_test[i]
for j in range(len(x_vel_test_sel)):
    Lx = Lx + (y_pred[0][j]/(((window_size-stride)/stride)))
    Ly = Ly + (y_pred[1][j]/(((window_size-stride)/stride)))
    pointx.append(Lx)
    pointy.append(Ly)
Pvx = pointx
Pvy = pointy  

print('Plotting Trajectory of length (meters): ',Cal_len_meters(Gvx, Gvy))

ptox = Pvx
ptoy = Pvy

plt.plot(Gvx,Gvy,label='Ground Truth',color='salmon')
plt.plot(ptox,ptoy,label='TinyOdom',color='green',linestyle='-')
plt.grid()
plt.legend(loc='best')
plt.title('Agrobot Localization')
plt.xlabel('East (m)')
plt.ylabel('North (m)')
plt.show()